In [3]:
# bug: NotImplementedError: Cannot convert a symbolic Tensor (simple_rnn/strided_slice:0) to a numpy array.
# fix: pip install numpy=1.19


In [1]:
# 導入函式庫
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Activation, Dense
from tensorflow.keras.optimizers import Adam

2024-10-19 16:13:50.946184: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-19 16:13:50.960754: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1729325630.973640   24491 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1729325630.977794   24491 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-19 16:13:50.992857: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# 載入 MNIST 資料庫的訓練資料，並自動分為『訓練組』及『測試組』
(X_train, y_train), (X_test, y_test_org) = mnist.load_data()

# 將 training 的 input 資料轉為3維，並 normalize 把顏色控制在 0 ~ 1 之間
X_train = X_train.reshape(-1, 28, 28) / 255.0
X_test = X_test.reshape(-1, 28, 28) / 255.0

In [3]:
# 建立簡單的線性執行的模型
model = Sequential()
# 加 RNN 隱藏層(hidden layer)
# 必須是 3 dimension
model.add(LSTM(units=256, input_shape=(28, 28)))

# 加 output 層
model.add(Dense(units=10, activation="softmax"))

I0000 00:00:1729325648.647616   24491 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9
/home/chesterxalan/github/Classwork/Classwork-PythonDLApplicationDevelopment/.venv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [4]:
# 編譯: 選擇損失函數、優化方法及成效衡量方式
LR = 0.001  # Learning Rate
adam = Adam(LR)
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 256)            │       291,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 294,410 (1.12 MB)

 Trainable params: 294,410 (1.12 MB)

 Non-trainable params: 0 (0.00 B)

None


In [5]:
# 將 training 的 label 進行 one-hot encoding，例如數字 7 經過 One-hot encoding 轉換後是 0000001000，即第7個值為 1
y_TrainOneHot = to_categorical(y_train)
y_TestOneHot = to_categorical(y_test_org)

# 將 training 的 input 資料轉為2維
X_train_2D = X_train.reshape(60000, 28, 28)
X_test_2D = X_test.reshape(10000, 28, 28)

x_Train_norm = X_train_2D / 255
x_Test_norm = X_test_2D / 255

In [6]:
# 進行訓練, 訓練過程會存在 train_history 變數中
train_history = model.fit(
    x=x_Train_norm,
    y=y_TrainOneHot,
    validation_split=0.2,
    epochs=10,
    batch_size=800,
    verbose=2,
)

Epoch 1/10


I0000 00:00:1729325667.071265   24636 cuda_dnn.cc:529] Loaded cuDNN version 90300


60/60 - 3s - 49ms/step - accuracy: 0.1369 - loss: 2.3089 - val_accuracy: 0.1211 - val_loss: 2.2986
Epoch 2/10
60/60 - 1s - 16ms/step - accuracy: 0.1564 - loss: 2.2990 - val_accuracy: 0.1354 - val_loss: 2.2975
Epoch 3/10
60/60 - 1s - 17ms/step - accuracy: 0.1925 - loss: 2.2860 - val_accuracy: 0.0995 - val_loss: 2.3207
Epoch 4/10
60/60 - 1s - 16ms/step - accuracy: 0.1963 - loss: 2.1857 - val_accuracy: 0.2279 - val_loss: 2.0773
Epoch 5/10
60/60 - 1s - 16ms/step - accuracy: 0.2313 - loss: 2.0605 - val_accuracy: 0.1437 - val_loss: 2.2349
Epoch 6/10
60/60 - 1s - 18ms/step - accuracy: 0.2816 - loss: 1.9625 - val_accuracy: 0.3697 - val_loss: 1.7048
Epoch 7/10
60/60 - 1s - 16ms/step - accuracy: 0.4203 - loss: 1.5849 - val_accuracy: 0.3874 - val_loss: 1.7053
Epoch 8/10
60/60 - 1s - 16ms/step - accuracy: 0.4452 - loss: 1.5240 - val_accuracy: 0.5042 - val_loss: 1.3541
Epoch 9/10
60/60 - 1s - 16ms/step - accuracy: 0.5211 - loss: 1.2955 - val_accuracy: 0.5431 - val_loss: 1.2872
Epoch 10/10
60/60 - 1

In [7]:
# 顯示訓練成果(分數)
loss, accuracy = model.evaluate(x_Test_norm, y_TestOneHot)
print(f"test loss: {loss}  test accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6050 - loss: 1.0913
test loss: 1.062315583229065  test accuracy: 0.6187000274658203


In [8]:
# 預測(prediction)
X = x_Test_norm[0:20, :]
predictions = np.argmax(model.predict(x_Test_norm[0:20]), axis=-1)
# get prediction result
print("actual :", y_test_org[0:20])
print("predict:", predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
actual : [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]
predict: [7 2 1 0 4 1 8 9 0 4 0 0 9 0 1 2 4 7 5 4]
